<a href="https://colab.research.google.com/github/hankiou/-Master-2-BI-Decision/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application BI: Aide à la décision
Enzo GUENY  
Adam SERGHINI  

*Sujet choisi: **Sujet 5** - Théorie du choix social*  
Trois profils de vote sont fournis. Le premier comporte 1000 électeurs et 9 candidats, les deux
autres comportent **10 000** électeurs et **12** candidats. Les données (accessibles vu la taille
uniquement en .csv) donnent **l’ordre de préférence** de chaque électeur pour les 12 candidats,
numérotés de 1 à 12.  
L'objectif est de programmer et de comparer les résultats obtenus par différents modes de scrutin possibles (vus en cours ou obtenus par recherche bibliographique).  
  
Nous effectuons les tests sur profil1, profil2, profil3 fournis en consigne, profil4 est un jeu de données extrait du TD *"examen 2"* fait en cours et nous sert de corpus de validation afin de comparer les résultats de nos implémentations avec les résultats obtenus en présentiel.


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from itertools import permutations
 
CORPUS_SIZE = 4 # How many files are executed

## Methods

### Tools
Méthodes utiles à l'implémentation des algorithmes de choix social

In [ ]:
def eliminate(matrix, candidate):
    shape = matrix.shape
    transpose = np.transpose(matrix)
    transpose = transpose[transpose != candidate]
    transpose = transpose.reshape(shape[1], shape[0] - 1)
    return np.transpose(transpose)

def getCandidates(matrix):
    return np.unique(matrix)

def executeDuels(matrix):
    candidates = getCandidates(matrix)
    shape = matrix.shape
    results = np.zeros((candidates.size, candidates.size), int)
    for candidate in candidates:
        for adversary in candidates:
            if(adversary != candidate):
                win = 0
                for i in range(0, shape[1]):
                    for j in range(0, shape[0]):
                        if(matrix[j][i] == adversary):
                            break
                        if(matrix[j][i] == candidate):
                            win = win +1
                            break

                results[candidate-1][adversary-1] = win
    return results

### Implementation
Implémentation des algorithmes de choix social vus en cours. Le principe de chaque méthode est décrit en commentaire dans le code source.

In [ ]:
# Selects elected candidate based on first preference
def relativeMajority(matrix, returnSize):
    return Counter(matrix[0]).most_common(returnSize)[0]

# Selects two winners for a first turn using relativeMajority()
# Eliminates every other candidate then processes a second turn among remaining candidates
def relativeMajorityT2(matrix):
    candidates = getCandidates(matrix)
    winners = [Counter(matrix[0]).most_common(2)[0][0], Counter(matrix[0]).most_common(2)[1][0]]
    eliminated = set(candidates) - set(winners)
    new_matrix = np.copy(matrix)
    for loser in eliminated:
        new_matrix[(new_matrix == loser)] = -1
        new_matrix = eliminate(new_matrix, -1)
    return Counter(new_matrix[0]).most_common(1)[0]
    
# Selects elected candidate based on borda method
# Score is calculated by adding (n - preference_order) for each voter
def borda(matrix):
    n = matrix.shape[0]
    scores = np.zeros(n).astype(np.int)
    voters = matrix.shape[1]
    for i in range (0, voters-1):
        for j in range (0, n-1):
            scores[matrix[j][i] -1] += (n - j)
    return scores.argmax()+1, scores.max()

# Returns the candidate winning the most duels against others
def condorcet(matrix):
    candidates = getCandidates(matrix)
    results = executeDuels(matrix)
    n = matrix.shape[1]
    winners = []
    w_duels = 0
    for candidate in candidates:
        duels = Counter(x >= n/2 for x in results[candidate-1])
        if(duels[True] > w_duels):
            winners = []
            w_duels = duels[True]
        if(w_duels == duels[True]):
            winners = np.append(winners, candidate)

    return winners, w_duels
    return 0

def kemenyYoung(matrix):
    candidates = getCandidates(matrix)
    best_score = 0
    best_ranking = None
    ketcha = executeDuels(matrix)
    for candidates_ranking in permutations(candidates):
        score = 0
        for candidate in range (0, candidates.size):
            for adversary in range (candidate, candidates.size): 
                score += ketcha[candidates_ranking[candidate]-1][candidates_ranking[adversary]-1]
        if score > best_score:
            best_score, best_ranking = score, candidates_ranking
    return (best_score, best_ranking)


# Eliminates the candidate being the least preffered each iteration
# until 2 remains, returns the most popular
def coombs(matrix):
    n = matrix.shape[0] -1
    new_matrix = np.copy(matrix)
    for i in range (0, n-1):
        unpopular =  Counter(new_matrix[new_matrix.shape[0] - 1]).most_common(1)[0][0]
        new_matrix = eliminate(new_matrix, unpopular)
    return Counter(new_matrix[0]).most_common(1)[0]

# Eliminates the candidate being the least top-voted each iteration
# until 2 remains, returns the most popular
def alternative(matrix):
    n = matrix.shape[0] -1
    new_matrix = np.copy(matrix)
    for i in range (0, n-1):
        unpopular =  Counter(new_matrix[0]).most_common(new_matrix.shape[0])[new_matrix.shape[0]-1][0]
        new_matrix = eliminate(new_matrix, unpopular)
    return Counter(new_matrix[0]).most_common(1)[0]

# Returns the candidate with the best score won on duelsagainst others minus lost duels
def copeland(matrix):
    candidates = getCandidates(matrix)
    results = executeDuels(matrix)
    n = matrix.shape[1]

    winners = []
    w_duels = 0
    for candidate in candidates:
        duels = Counter(x >= n/2 for x in results[candidate-1]) - Counter(x < n/2 for x in results[candidate-1])
        if(duels[True] > w_duels):
            winners = []
            w_duels = duels[True]
        if(w_duels == duels[True]):
            winners = np.append(winners, candidate)

    return winners, w_duels
    return 0


## Data

In [ ]:
# Load Data
# data/choix_social/

matrices = [None] * CORPUS_SIZE # Stores matrix conversions of each file
for i in range (0, CORPUS_SIZE):
    filename = 'data/choix_social/profil'+ str(i+1) + '.csv'
    data = pd.read_csv(filename, header=None)
    matrices[i] = data.to_numpy()
    print("Profil" + str(i+1) + ": " + str(matrices[i].shape))

Profil1: (9, 1000)
Profil2: (12, 10000)
Profil3: (12, 10000)
Profil4: (4, 100)


*TODO: Commentaire sur la data*  

||Profil1|Profil2|Profil3|Profil4|
|:-|:-:|:-:|:-:|:-:|
|Nombre de candidats|9|12|12|4|
|Nombre de votants|1,000|10,000|10,000|100|

In [ ]:
# DEV SANDBOX


## Execution

In [ ]:
#Relative Majority 1 turn
print("Relative Majority 1 turn:")
for i in range (0, CORPUS_SIZE):
    result = relativeMajority(matrices[i], 1)
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

#Relative Majority 2 turns
print("\nRelative Majority 2 turns:")
for i in range (0, CORPUS_SIZE):
    result = relativeMajorityT2(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

# Condorcet
print("\nCondorcet :")
for i in range (0, CORPUS_SIZE):
    result = condorcet(matrices[i])
    print("Profil" + str(i+1) + ": Candidats " + str(result[0]) + ": " + str(result[1]) + " duels gagnés")


# Borda
print("\nBorda :")
for i in range (0, CORPUS_SIZE):
    result = borda(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + "pts")

# Coombs
print("\nCoombs :")
for i in range (0, CORPUS_SIZE):
    result = coombs(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix reportées (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

# Alternative
print("\nAlternative :")
for i in range (0, CORPUS_SIZE):
    result = alternative(matrices[i])
    print("Profil" + str(i+1) + ": Candidat " + str(result[0]) + ": " + str(result[1]) + " voix reportées (" +  "{:.2f}".format(result[1]/matrices[i][0].size*100) + " %)")

# Copeland
print("\nCopeland :")
for i in range (0, CORPUS_SIZE):
    result = copeland(matrices[i])
    print("Profil" + str(i+1) + ": Candidats " + str(result[0]) + ": " + str(result[1]) + " duels gagnés (- duels perdus)")

# KemenyYoung
print("\nKemeny Young :")
for i in range(0, CORPUS_SIZE):
    result = kemenyYoung(matrices[i])
    print("Profil"+ str(i+1) + ": Classement" + str(result[1]) + ": " + str(result[0]) + "pts")




Relative Majority 1 turn:
Profil1: Candidat 4: 293 voix (29.30 %)
Profil2: Candidat 10: 1828 voix (18.28 %)
Profil3: Candidat 7: 1498 voix (14.98 %)
Profil4: Candidat 1: 35 voix (35.00 %)

Relative Majority 2 turns:
Profil1: Candidat 4: 564 voix (56.40 %)
Profil2: Candidat 5: 6366 voix (63.66 %)
Profil3: Candidat 7: 5078 voix (50.78 %)
Profil4: Candidat 4: 65 voix (65.00 %)

Condorcet :
Profil1: Candidats [5.]: 8 duels gagnés
Profil2: Candidats [7.]: 11 duels gagnés
Profil3: Candidats [ 7.  8. 11.]: 10 duels gagnés
Profil4: Candidats [3.]: 3 duels gagnés

Borda :
Profil1: Candidat 5: 6451pts
Profil2: Candidat 7: 95904pts
Profil3: Candidat 6: 80155pts
Profil4: Candidat 3: 306pts

Coombs :
Profil1: Candidat 5: 524 voix reportées (52.40 %)
Profil2: Candidat 7: 5398 voix reportées (53.98 %)
Profil3: Candidat 11: 5712 voix reportées (57.12 %)
Profil4: Candidat 3: 75 voix reportées (75.00 %)

Alternative :
Profil1: Candidat 4: 564 voix reportées (56.40 %)
Profil2: Candidat 7: 5398 voix repor

## Interprétation des Résultats
### Profil 1
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative| Copeland
|:-|:-|:-|:-|:-|:-|:-|:-|
|**4** (293 - *29.30%*)|**4** (564 - *56.40%*)|**5** (8)|**5** (6,451 pts)|**5** (524 - *52.40%*)|{5, 4, 7, 8, 6, 3, 9, 2, 1} (24597 pts)  |**4** (564 - *56.40%*)|**5** (7)|
### Profil 2
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|
|:-|:-|:-|:-|:-|:-|:-|
|**10** (1828 - *18.28%*)|**5** (6366 - *63.66%*)|**7** (11)|**7** (95,904 pts)|**7** (5398 - *53.98%*)|-|**7** (5398 - *53.98%*)|**7** (10)|
### Profil 3
|Majorité Relative (1 tour)|Majorité Relative (2 tours)|Condorcet|Borda|Coombs|Kemeny-Young|Alternative|
|:-|:-|:-|:-|:-|:-|:-|
|**7** (1498 - *14.97%*)|**7** (5078 - *50.78%*)|**7 8 11** (10)|**6** (80,155 pts)|**5** (5712 - *57.12%*)|-|**6** (5010 - *50.10%*)|**7 8 11** (8)|